In [31]:
from graphdatascience import GraphDataScience

In [32]:
NEO4J_URI = "bolt://localhost:7688"
NEO4J_AUTH = ("neo4j","birglab1")
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

# Necessary if you enabled Arrow on GDS of the targeted DB. Override with your DB name
NEO4J_DB = "neo4j"
gds.set_database(NEO4J_DB)

In [126]:
"""
  'persons',
  'MATCH (n:Person) RETURN id(n) AS id',
  'MATCH (n:Person)-[r:KNOWS]->(m:Person) RETURN id(n) AS source, id(m) AS target')
  """

"""CALL gds.graph.project.cypher(
  'readWithProperties',
  'MATCH (n) RETURN id(n) AS id, labels(n) AS labels',
  'MATCH (n)-[r:READ]->(m)
    RETURN id(n) AS source, id(m) AS target, type(r) AS type, r.numberOfPages AS numberOfPages'
)"""
G,_ = gds.graph.project.cypher("linkage_g9",
                             "match (p) where p:Paper or p:Resource return id(p) as id, labels(p) AS labels, toInteger(p.fear) as fear, toInteger(p.biomechanics) as biomechanics, p.classification as classification",
                             "match (p:Paper)-[r:HAS_ENTITY]->(e:Resource) return id(p) as source, id(e) as target, type(r) AS type")


Loading:   0%|          | 0/100 [00:00<?, ?%/s]

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project.cypher`: Caused by: java.lang.IllegalArgumentException: Unsupported GDS node property of type `String`.}

In [127]:
G

Graph({'graphName': 'linkage_g8', 'nodeCount': 349808, 'relationshipCount': 4750, 'database': 'neo4j', 'configuration': {'relationshipQuery': 'match (p:Paper)-[r:HAS_ENTITY]->(e:Resource) return id(p) as source, id(e) as target, type(r) AS type', 'jobId': '85ad5eab-7ec5-4346-bdf5-a8b7321c882e', 'creationTime': neo4j.time.DateTime(2023, 5, 8, 16, 23, 15, 746388000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'validateRelationships': True, 'nodeQuery': 'match (p) where p:Paper or p:Resource return id(p) as id, labels(p) AS labels, toInteger(p.fear) as fear, toInteger(p.biomechanics) as biomechanics', 'logProgress': True, 'sudo': True, 'readConcurrency': 4, 'parameters': []}, 'schema': {'graphProperties': {}, 'relationships': {'HAS_ENTITY': {}}, 'nodes': {'ObjectProperty': {}, 'Ontology': {}, 'TransitiveProperty': {}, 'SelResourceC0026857': {}, 'Resource': {}, 'FunctionalProperty': {}, 'PaperFearBiomechanics': {'biomechanics': 'Integer (DefaultValue(-922337203685477

In [129]:
pipe, _ = gds.beta.pipeline.nodeClassification.create("classification-predictor-pipe9")

In [130]:
_ = pipe.configureSplit(testFraction=0.796)

In [131]:
_ = pipe.addNodeProperty(
    "beta.hashgnn",
    mutateProperty="embedding",
    iterations=4,
    heterogeneous=True,
    embeddingDensity=512,
    neighborInfluence=0.7,
    featureProperties=["fear","biomechanics"],
    randomSeed=41,
    contextNodeLabels=["Paper"]
)

In [132]:
_ = pipe.selectFeatures("embedding")

In [133]:
_ = pipe.addLogisticRegression(penalty=(0.1, 1.0), maxEpochs=1000, patience=5, tolerance=0.0001, learningRate=0.01)

In [134]:
_ = pipe.addRandomForest(minSplitSize=(2, 100), criterion="ENTROPY")

In [136]:
model, _ = pipe.train(
    G,
    modelName="genre-predictor-model3",
    targetNodeLabels=["Paper"],
    targetProperty="classification",
    metrics=["F1_MACRO"],
    randomSeed=42,
)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.beta.pipeline.nodeClassification.train`: Caused by: java.lang.IllegalArgumentException: Target property `classification` not found in graph with node properties: [biomechanics, fear]}

In [121]:
print(f"Accuracy scores of trained model:\n{model.metrics()['F1_MACRO']}")

Accuracy scores of trained model:
(test=0.9999999944505494, outer_train=0.9999999929166666, train=(min=0.99999999125, avg=0.9999999918055554, max=0.9999999920833333), validation=(min=0.33333333000000004, avg=0.7777777683333333, max=0.9999999875000001))


In [107]:
gds.beta.hashgnn.stream('linkage_g5',
  {
    'nodeLabels': ['Paper'],
    'iterations': 1,
    'embeddingDensity': 2,
    featureProperties: ['fear', 'biomechanism'],
    'randomSeed: 42
  }
)

TypeError: __call__() takes 2 positional arguments but 3 were given